In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,"
import hydra
from omegaconf import DictConfig

from trainer import Trainer

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path="../config", job_name=".")
cfg = compose(config_name="trainer", overrides=["env.train.id=BreakoutNoFrameskip-v4", "wandb.mode=online"])

In [6]:
trainer=Trainer(cfg)

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


Tokenizer : shape of latent is (512, 4, 4).
18743683 parameters in agent.tokenizer
8447493 parameters in agent.world_model
3217957 parameters in agent.actor_critic


In [9]:
to_log=trainer.train_collector.collect(trainer.agent, 2, **trainer.cfg.collection.train.config)

Experience collection (train_dataset): 100%|██████████| 200/200 [00:08<00:00, 23.49it/s]


In [10]:
trainer.cfg.common.sequence_length

20

In [85]:
import torch

all_but_last_obs_tokens_pattern = torch.ones(trainer.cfg.world_model.tokens_per_block)
all_but_last_obs_tokens_pattern[-2] = 0
act_tokens_pattern = torch.zeros(trainer.cfg.world_model.tokens_per_block)
act_tokens_pattern[-1] = 1
obs_tokens_pattern = 1 - act_tokens_pattern

In [130]:
trainer.cfg.training.world_model

{'batch_num_samples': 64, 'grad_acc_steps': 1, 'max_grad_norm': 10.0, 'weight_decay': 0.01, 'start_after_epochs': 25, 'steps_per_epoch': 200}

In [11]:
batch = trainer.train_dataset.sample_batch(64, 20, [0.125,0.125,0.25,0.5], True)

In [45]:
trainer.cfg.env.train
trainer.cfg.collection.train.num_envs

({'_target_': 'envs.make_atari', 'id': 'BreakoutNoFrameskip-v4', 'size': 64, 'max_episode_steps': 20000, 'noop_max': 30, 'frame_skip': 4, 'done_on_life_loss': True, 'clip_reward': False},
 1)

In [28]:
import torch
from einops import rearrange
with torch.no_grad():
    obs_tokens = trainer.agent.tokenizer.encode(batch['observations'].cuda(), should_preprocess=True).tokens  # (BL, K)
act_tokens = rearrange(batch['actions'], 'b l -> b l 1').cuda()
tokens = rearrange(torch.cat((obs_tokens, act_tokens), dim=2), 'b l k1 -> b (l k1)')  # (B, L(K+1))

outputs = trainer.agent.world_model(tokens)

In [34]:
print(outputs.logits_observations.shape)
logits_observations = rearrange(outputs.logits_observations[:, :-1], 'b t o -> (b t) o')
print(logits_observations.shape)

torch.Size([64, 320, 512])
torch.Size([20416, 512])


In [35]:
outputs.logits_observations[:, :-1].shape

torch.Size([64, 319, 512])

In [23]:
batch['observations'].cpu()
obs_tokens=obs_tokens.cpu()
labels_observations, labels_rewards, labels_ends=trainer.agent.world_model.compute_labels_world_model(obs_tokens, batch['rewards'], batch['ends'], batch['mask_padding'])

In [24]:
labels_observations.shape

torch.Size([20416])